In [1]:
# standarad imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import unicodedata

# import regular expression operations
import re

# import natural language toolkit
import nltk
# import stopwords list
from nltk.corpus import stopwords
# import wordcloud
from wordcloud import WordCloud

import requests
from bs4 import BeautifulSoup
import os
from pprint import pprint 
import time
import json

import acquire as a
import wrangle as w

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression



# acquire

In [15]:
data = pd.read_json('github_data.json')
data

,repo,language,readme_contents
0,/nyukat/breast_cancer_classifier,Jupyter Notebook,# Deep Neural Networks Improve Radiologists' P...
1,/lishen/end2end-all-conv,Jupyter Notebook,Shield: [![CC BY-NC-SA 4.0][cc-by-nc-sa-shield...
2,/ImagingLab/ICIAR2018,Python,# ICIAR2018\n### Two-Stage Convolutional Neura...
3,/Jean-njoroge/Breast-cancer-risk-prediction,Jupyter Notebook,# Breast-cancer-risk-prediction\n\n> Necessity...
4,/abhinavsagar/breast-cancer-classification,Jupyter Notebook,# Breast-cancer-classification\n\nBreast Cance...
...,...,...,...
90,/BishalDali/Breast_Cancer_Prediction,Jupyter Notebook,# Breast_Cancer_Prediction
91,/IndianAIProduction-Channel/Breast-Cancer-Dete...,HTML,# Breast-Cancer-Detection-App\n Breast Cancer ...
92,/gmineo/Breast-Cancer-Prediction-Project,R,# Breast-Cancer-Prediction\n\nChoose-your-own ...
93,/iharnoor/BreastCancer-Kmeans,Python,# BreastCancer-Kmeans\nClustering the data int...


In [3]:
df = pd.DataFrame(data)

# Prepare!

In [4]:
#df = df.dropna()

In [5]:
# text cleaning on a df column named 'readme'. removing special character and words.
df.readme_contents = df.readme_contents.str.replace('[/,_,-,:,"]', '', regex=True)

df.readme_contents = df.readme_contents.str.replace('heavy', '').str.replace('check', '').str.replace('mark', '')

In [6]:
df.language.value_counts()


language
Python              45
Jupyter Notebook    29
R                    4
HTML                 4
MATLAB               3
TeX                  2
C#                   2
Matlab               2
Objective-C          1
Dart                 1
Name: count, dtype: int64

In [8]:
df['clean_norm_token'] = df.readme_contents.apply(w.tokenize).apply(w.basic_clean).apply(w.remove_stopwords).apply(w.lemmatize)

# tokenizes, cleans, remove stopwprds, lemmatize the text in the readme_contents and 
#stores the processed text in a new column clean_norm_token

df.clean_norm_token = df.clean_norm_token.str.replace('124', '')

# it removes any occurences of the string '124' from the clean_norm_token column

In [9]:
#in language column replace language with other if it is not in the top 3 languages
top_5 = df.language.value_counts().head(5).index.tolist()
print(top_5)
df.language = df.language.apply(lambda x: x if x in top_5 else 'other')

['Python', 'Jupyter Notebook', 'R', 'HTML', 'MATLAB']


In [10]:
df.language.value_counts()


language
Python              45
Jupyter Notebook    29
other               10
R                    4
HTML                 4
MATLAB               3
Name: count, dtype: int64

In [13]:
#baseline is 47%
df.language.value_counts(normalize=True)
#the relative frequency of each programming language in  dataset.

language
Python              0.473684
Jupyter Notebook    0.305263
other               0.105263
R                   0.042105
HTML                0.042105
MATLAB              0.031579
Name: proportion, dtype: float64

In [16]:
#calls the split function from wrangle.py
train, validate, test = w.split_function_cat_target(df, "language")